In [ ]:
%pip install aiohttp

# Streaming Document Submission

This notebook implements streaming document processing with Acordis API submission and Guava rate limiting.

## Key Features:

- **Structured Streaming** with foreachBatch for scalability
- **Guava RateLimiter** for API throttling
- **Fault tolerance** with retry logic
- **Delta Lake** for submission tracking


## Imports

In [ ]:

import json
import time
import uuid
from datetime import datetime
import pandas as pd
import traceback
# Import required libraries
from pyspark.sql import functions as F
from pyspark.sql.types import StructType, StructField, StringType, BinaryType, LongType, TimestampType, IntegerType

# TODO: Guava RateLimiter equivalent in Python

# import sys
# sys.path.append("../../../src")

# Import unified Acordis client from framework
# from framework.acordis_client import AcordisApiClient
# # Import ConfigManager from framework
# from framework.utils.config import ConfigManager


## ConfigManager

TODO: Move to framework once packaging issue is resolved

In [ ]:
"""
Configuration management utilities for PDF processing framework.

This module provides utilities for loading, validating, and managing
configuration files used throughout the PDF processing pipeline.
"""

import yaml
import os
from typing import Dict, Any, List, Optional, Union
from pathlib import Path


class ConfigManager:
    """
    Configuration manager for handling YAML and JSON configuration files.
    
    Provides methods for:
    - Loading configuration from files
    - Validating configuration structure
    - Merging configurations
    - Environment variable substitution
    """
    
    def __init__(self, config_path: Optional[Union[str, Path]] = None):
        """
        Initialize configuration manager.
        
        Args:
            config_path: Path to configuration file
            config_dict: Configuration dictionary (alternative to file)
            
        Raises:
            ValueError: If neither config_path nor config_dict is provided
        """
        if config_path:
            self.config = self.load_config_file(config_path)
            self.config_path = str(config_path)
        else:
            raise ValueError("Either config_path or config_dict must be provided")
        
        # Substitute environment variables
        # self._substitute_environment_variables()

    @staticmethod
    def get_file_extension(file_path: Union[str, Path]) -> str:
        """
        Get file extension (without dot).
        
        Args:
            file_path: Path to file
            
        Returns:
            File extension in lowercase
        """
        return Path(file_path).suffix.lower().lstrip('.')
    
    @staticmethod
    def load_config_file(file_path: Union[str, Path]) -> Dict[str, Any]:
        """
        Load configuration from YAML or JSON file.
        
        Args:
            file_path: Path to configuration file
            
        Returns:
            Configuration dictionary
            
        Raises:
            FileNotFoundError: If file doesn't exist
            ValueError: If file format is not supported
        """
        extension = ConfigManager.get_file_extension(file_path)
        
        if extension in ['yaml', 'yml']:
            with open(file_path, 'r', encoding='utf-8') as f:
                return yaml.safe_load(f) or {}
        else:
            raise ValueError(f"Unsupported configuration file format: {extension}")
    
    def get(self, key: str, default: Any = None) -> Any:
        """
        Get configuration value with optional default.
        
        Args:
            key: Configuration key (supports dot notation for nested keys)
            default: Default value if key is not found
            
        Returns:
            Configuration value or default
        """
        return self._get_nested_value(self.config, key, default)
        
    @staticmethod
    def _get_nested_value(config: Dict[str, Any], key: str, default: Any = None) -> Any:
        """
        Get nested configuration value using dot notation.
        
        Args:
            config: Configuration dictionary
            key: Key with dot notation (e.g., 'task.params.value')
            default: Default value if key is not found
            
        Returns:
            Configuration value or default
        """
        keys = key.split('.')
        value = config
        
        for k in keys:
            if isinstance(value, dict) and k in value:
                value = value[k]
            else:
                return default
        
        return value
        
    def to_dict(self) -> Dict[str, Any]:
        """
        Get configuration as dictionary.
        
        Returns:
            Configuration dictionary
        """
        return self.config.copy()
    

    def get_pipeline_steps(config: dict, pipeline_name: str, document_format: str, step_name: str) -> dict:
        """
        Return the steps dictionary for a given pipeline and document format.
        """
        step_list = config.get(pipeline_name, {}).get(document_format, [])
        step_config = ConfigManager.get_step_config(step_list, step_name)
        return step_config


    @staticmethod
    def get_step_config(config: list, step_name: str) -> dict:
        """
        Fetch all variables for a given step from the workflow config.

        Args:
            config (list): List of step dictionaries.
            step_name (str): Step name to search for.

        Returns:
            dict: Config dictionary for the step, or {} if not found.
        """
        for step in config:
            if step.get("step") == step_name:
                return step
        return {}


## Get task variables

In [ ]:

print("✓ Basic imports completed")
environment = dbutils.jobs.taskValues.get(taskKey="unified_workflow_setup", key="environment")
job_id = dbutils.jobs.taskValues.get(taskKey="unified_workflow_setup", key="databricks_job_id")
job_run_id = dbutils.jobs.taskValues.get(taskKey="unified_workflow_setup", key="databricks_job_run_id")

globals_config = json.loads(dbutils.jobs.taskValues.get(taskKey="unified_workflow_setup", key="globals_config"))
services_config = json.loads(dbutils.jobs.taskValues.get(taskKey="unified_workflow_setup", key="services_config"))
pipelines_config = json.loads(dbutils.jobs.taskValues.get(taskKey="unified_workflow_setup", key="pipelines_config"))
unified_steps_config = json.loads(dbutils.jobs.taskValues.get(taskKey="unified_workflow_setup", key="unified_steps_config"))

# get global configs
catalog = globals_config["catalog"]
bronze_schema_name = globals_config["bronze_schema_name"]
max_files_per_trigger = globals_config["default_batch_size"]
checkpoint_base_location = globals_config["checkpoint_base_location"]
checkpoint_location = f"{checkpoint_base_location}/submission_checkpoints"

# get services configs
acordis_config = services_config["acordis_service"]
api_url = acordis_config['api_url']
timeout = acordis_config['timeout_seconds']

# get pipeline config
# submission_cfg = ConfigManager.get_pipeline_steps(pipelines_config, pipeline_id, "pdf", "SubmissionProcessor") 
# output_submission_status = submission_cfg["submission_status"]
# landing_zone = submission_cfg["landing_zone"]
# output_table = submission_cfg["output_table"]
# enableChangeDataFeed = submission_cfg["enableChangeDataFeed"]
# synthetic_mode = submission_cfg["synthetic_mode"]
# pathGlobFilter = submission_cfg["pathGlobFilter"]

# get unified steps config
unified_pdf_config = unified_steps_config["submission"]
output_submission_status = unified_pdf_config["submission_status"]
landing_zone = unified_pdf_config["landing_zone"]
output_table = unified_pdf_config["output_table"]
enableChangeDataFeed = unified_pdf_config["enableChangeDataFeed"]
synthetic_mode = unified_pdf_config["synthetic_mode"]
pathGlobFilter = unified_pdf_config["pathGlobFilter"]
acodis_file_path = unified_pdf_config["acodis_file_path"]
unstructured_file_path = unified_pdf_config["unstructured_file_path"]


# Define explicit schema
result_schema = StructType([
    StructField("submission_id", StringType(), True),
    StructField("pipeline_id", StringType(), True), 
    StructField("document_format", StringType(), True),
    StructField("submission_status", StringType(), True),
    StructField("file_path", StringType(), True),
    StructField("file_size_bytes", LongType(), True),
    StructField("file_modification_time", StringType(), True),
    StructField("submission_timestamp", TimestampType(), True),
    StructField("last_updated", TimestampType(), True),
    StructField("submission_databricks_job_id", StringType(), True),
    StructField("submission_databricks_run_id", StringType(), True),
    StructField("api_transaction_id", StringType(), True),
    StructField("api_status", StringType(), True),
    StructField("api_response_time_ms", LongType(), True),
    StructField("api_response", StringType(), True),
    StructField("error_message", StringType(), True)
])


In [ ]:
%md ##Setup Functions

In [ ]:
import requests

def get_acodis_session(acordis_config):
    """returns a session for accodis"""

    token = acordis_config["api_key"]
    session = requests.Session()
    session.headers.update({
                "Authorization": f"Bearer {token}",
                "User-Agent": "Databricks-DocumentProcessor/1.0"
            })
    
    return session

session = get_acodis_session(acordis_config)



## Get Acordis Client

In [ ]:
def execute_document_submission_step(submission_id: str, file_path: str, priority: int = 1, use_synthetic: bool = False) -> dict:
    
    import hashlib
    import time
    """
    Submit a document for processing.
    
    Args:
        file_path: Path to the document file
        document_id: Unique identifier for the document
        priority: Processing priority (1-10, higher is more urgent)
        
    Returns:
        Standardized response dictionary with success status and details
    """
    try:
        start_time = time.time()
        payload = {}
        if use_synthetic:
            # SYNTHETIC MODE: Mock API response for testing
            time.sleep(0.1)  # Simulate processing time
            response_time_ms = int((time.time() - start_time) * 1000)
            
            # Generate realistic synthetic transaction ID
            hash_input = f"{submission_id}{file_path}{time.time()}"
            transaction_id = f"txn_{hashlib.md5(hash_input.encode()).hexdigest()[:12]}"
            
            # Simulate successful response matching Acordis API structure
            synthetic_response = {
                "transaction_id": transaction_id,
                "status": "submitted",
                "message": "File successfully submitted for processing",
                "file_info": {
                    "name": file_path.split('/')[-1],
                    "size": "unknown",
                    "type": "pdf"
                },
                "processing_info": {
                    "estimated_completion": "2024-01-01T12:00:00Z",
                    "priority": priority
                }
            }

            payload = {
                'success': True,
                'transaction_id': transaction_id,
                'status': 'submitted',
                'response_time_ms': response_time_ms,
                'full_response': json.dumps(synthetic_response)
            }
            
            return payload
        else:
            # PRODUCTION MODE: Real API call to Acordis
            # Read file content (assuming PDF)
            file_path = file_path[5:]
            with open(file_path, 'rb') as file:
                file_content = file.read()
            
            # Prepare multipart form data according to API specification
            files = {
                'files': (file_path.split('/')[-1], file_content, 'application/pdf')
            }
            
            # Make actual API call to /api/v1/transaction endpoint
            response = session.post(
                f"{api_url}/api/v1/transaction",
                files=files,
                timeout=timeout
            )
            
            response_time_ms = int((time.time() - start_time) * 1000)
            
            # Handle response based on Acordis API status codes: 200, 400, 401, 403, 422
            if response.status_code == 200:
                # Successful submission
                api_response = response.json()
                api_response_string = json.dumps(api_response)
                return {
                    'success': True,
                    'transaction_id': api_response[0].get('transactionId'),
                    'status': 'submitted',
                    'response_time_ms': response_time_ms,
                    'full_response': api_response_string
                }
            elif response.status_code == 400:
                # Bad Request - invalid file format, missing required fields, etc.
                error_msg = f"Bad Request (400): Invalid file or missing required fields"
                try:
                    error_details = response.json()
                    error_msg += f" - {error_details}"
                except:
                    error_msg += f" - {response.text}"
                
                return {
                    'success': False,
                    'transaction_id': None,
                    'error': error_msg,
                    'response_time_ms': response_time_ms,
                    'full_response': response.text
                }
            elif response.status_code == 401:
                # Unauthorized - invalid API key
                return {
                    'success': False,
                    'transaction_id': None,
                    'error': "Unauthorized (401): Invalid API key or authentication failed",
                    'response_time_ms': response_time_ms,
                    'full_response': response.text
                }
            elif response.status_code == 403:
                # Forbidden - insufficient permissions
                return {
                    'success': False,
                    'transaction_id': None,
                    'error': "Forbidden (403): Insufficient permissions for this operation",
                    'response_time_ms': response_time_ms,
                    'full_response': response.text
                }
            elif response.status_code == 422:
                # Unprocessable Entity - validation errors
                error_msg = f"Validation Error (422): File failed validation checks"
                try:
                    error_details = response.json()
                    error_msg += f" - {error_details}"
                except:
                    error_msg += f" - {response.text}"
                
                return {
                    'success': False,
                    'transaction_id': None,
                    'error': error_msg,
                    'response_time_ms': response_time_ms,
                    'full_response': response.text
                }
            else:
                # Other unexpected status codes
                return {
                    'success': False,
                    'transaction_id': None,
                    'error': f"Unexpected API error {response.status_code}: {response.text}",
                    'response_time_ms': response_time_ms,
                    'full_response': response.text
                }
                    
                
    except Exception as e:
        payload = {
            'success': False,
            'transaction_id': None,
            'error': f"Document submission error: {str(e)}",
            'response_time_ms': int((time.time() - start_time) * 1000) if 'start_time' in locals() else 0,
            'full_response': None
        }
        return payload


In [ ]:
# --- PARTITION PROCESSOR ---
def process_partition(rows_iter):
    """
    Runs on executors. 
      1) Read data from volumes
      2) Create a submission record with status 'submitted'
      3) Submit the document to Acordis or unstructured
      4) Update the submission record with the API transaction ID, status, and response time
    """
    for row_df in rows_iter:
        out_rows = []
        cols = result_schema.names 
        if row_df.empty:
            yield pd.DataFrame(columns=cols)
            continue

        for _, row in row_df.iterrows():

            file_path = row.path
            file_size = row.length
            file_mod_time = str(row.modificationTime)
            # Generate document ID
            submission_id = str(uuid.uuid4())
            new_submission_status = ""
            api_transaction_id = None
            api_status = None
            api_response_time_ms = None
            api_response = None
            error_message = None
            pipeline_id = "acordis"
            print(f"Processing file: {file_path}")

            # Submit to Acordis API with retries
            submission_result = None

            try:
                # acodis processing
                # get second to last element for the nested folder path
                # file_path is "/Volume/path/to/folder/acodis/filename.pdf"
                # -2 gives if its acodis or unstructured
                if file_path.split("/")[-2] == acodis_file_path:
                    pipeline_id = "acodis"
                    submission_result = execute_document_submission_step(submission_id, file_path, use_synthetic=synthetic_mode)

                    # acordis_client.submit_document(file_path, submission_id)
                    if submission_result['success']:
                        new_submission_status = "submitted"   
                        api_transaction_id = submission_result['transaction_id']                
                        api_status = 'submitted' 
                        api_response_time_ms = submission_result['response_time_ms']
                        api_response = submission_result['full_response']
                        error_message = None
                    else:
                        new_submission_status = "failed"   
                        api_transaction_id = submission_result['transaction_id']                
                        api_status = 'failed'
                        api_response_time_ms = submission_result['response_time_ms']
                        api_response = submission_result['full_response']
                        error_message = submission_result['error']

                # unstructured processing
                # get second to last element for the nested folder path
                # file_path is "/Volume/path/to/folder/unstructured/filename.pdf"
                # -2 gives if its acodis or unstructured
                elif file_path.split("/")[-2] == unstructured_file_path:
                    pipeline_id = "unstructured"
                    new_submission_status = "submitted"   
                    api_transaction_id = 'N/A'
                    api_status = 'N/A'
                    api_response_time_ms = None
                    api_response = 'N/A'
                    error_message = None
                
                # raise error (this will be caught)
                else:
                    raise NotImplementedError("file processing is unknow; should be Acodis or Unstructured")

            except Exception as e:
                pipeline_id = "unknown"
                new_submission_status = "failed"
                api_transaction_id = None               
                api_status = 'error' 
                api_response_time_ms = None
                api_response = None
                error_message = traceback.format_exc()         
                
            # Create tracking record
            processed_record = {
                'submission_id': submission_id,
                'pipeline_id': pipeline_id,
                'document_format': 'pdf', # TODO: Change this to the actual format
                'submission_status': new_submission_status,            
                'file_path': file_path,
                'file_size_bytes': file_size,
                'file_modification_time': file_mod_time,
                'submission_timestamp': datetime.now(),                
                'last_updated': datetime.now(),
                'submission_databricks_job_id': job_id,
                'submission_databricks_run_id': job_run_id,
                'api_transaction_id': api_transaction_id,
                'api_status': api_status,
                'api_response_time_ms': api_response_time_ms,
                'api_response': api_response,
                'error_message': error_message
            }

            out_rows.append(processed_record)

        yield pd.DataFrame(out_rows, columns=cols)


In [ ]:
# Clean batch processing function for testing serialization
def foreach_batch(batch_df, batch_id):
    """Simple test function to verify Spark Connect serialization works"""

    if batch_df.count() == 0:
        print(f"→ Batch {batch_id}: empty")
        return
    
    print(f"processing {batch_df.count()} records with id : {batch_id}")

    result_df = batch_df.mapInPandas(process_partition, schema=result_schema)

    # add batch id to result
    result_df = result_df.withColumn("batch_id", F.lit(batch_id).cast("string"))

    print(f"map in pandas processing complete and appended {result_df.count()} records")
    
    (result_df.write
     .format("delta")
     .mode("append")
     .option("mergeSchema", "true")
     .option("delta.enableChangeDataFeed", enableChangeDataFeed)
     .saveAsTable(f"{catalog}.{bronze_schema_name}.{output_table}"))



## Read Source Data
Read data from Volumes using Autoloader and create a submission entry

In [ ]:
# Configure and start streaming job
from datetime import datetime
print("🚀 Starting streaming document submission...")

file_stream = (spark.readStream
               .format("cloudFiles")
               .option("cloudFiles.format", "binaryFile")
               .option("cloudFiles.maxFilesPerTrigger", max_files_per_trigger)
               .option("cloudFiles.includeExistingFiles", "true")
               # .option("cloudFiles.recursiveFileLookup", "true")
               .option("pathGlobFilter", pathGlobFilter)
               .load(landing_zone))


## Write Data 
1. Write data to bronze submissions table
2. On success, update submission_status=submitted

In [ ]:
# Start the streaming job with simple test function
streaming_query = (file_stream.writeStream
                   .foreachBatch(foreach_batch)
                   .option("checkpointLocation", checkpoint_location)
                   .trigger(availableNow=True)
                   .outputMode("append")
                   .start())

print("Document Submission Job Started")

try:
    streaming_query.awaitTermination()
    print("Document Submission Job completed successfully")    
    
except Exception as e:
    print(f"Streaming job error: {str(e)}")
    streaming_query.stop()
    raise                

